<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Darwin':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)
df.head()

/var/folders/k9/vb534jt106d2plphjzl86xcw0000gn/T/ipykernel_51533/3634877419.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)


,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6812


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

2023-11-20 20:58:45.917490: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['look',
  'i',
  'was',
  'gonna',
  'go',
  'easy',
  'on',
  'you',
  'and',
  'not',
  'to',
  'hurt',
  'your',
  'feelings'],
 ['but', "i'm", 'only', 'going', 'to', 'get', 'this', 'one', 'chance']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6812


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1347


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 506556.96875
Loss after epoch 1: 369829.53125
Loss after epoch 2: 344129.125
Loss after epoch 3: 314718.375
Loss after epoch 4: 310399.0
Loss after epoch 5: 295839.5
Loss after epoch 6: 262763.5
Loss after epoch 7: 258724.5
Loss after epoch 8: 254935.0
Loss after epoch 9: 252422.0
Loss after epoch 10: 249889.75
Loss after epoch 11: 246938.0
Loss after epoch 12: 243465.0
Loss after epoch 13: 241613.5
Loss after epoch 14: 229063.75
Loss after epoch 15: 226176.0
Loss after epoch 16: 225148.5
Loss after epoch 17: 223591.5
Loss after epoch 18: 222550.5
Loss after epoch 19: 222643.0


(785273, 1296760)

### 4 - Ensayar

In [28]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('food', 0.7980446815490723),
 ('stamps', 0.7704891562461853),
 ('damn', 0.7608792781829834),
 ('ooh', 0.7356470823287964),
 ('swear', 0.7144322395324707),
 ('thank', 0.7053040862083435),
 ('yea', 0.6808665990829468),
 ('ah', 0.6683138608932495),
 ('sent', 0.6661420464515686),
 ('acid', 0.656876802444458)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["feelings"], topn=10)

[('his', -0.1854957640171051),
 ('he', -0.24228157103061676),
 ("he's", -0.24640122056007385),
 ('these', -0.2698219418525696),
 ('o', -0.27692732214927673),
 ('are', -0.2918413281440735),
 ('been', -0.2932719588279724),
 ('of', -0.3024033010005951),
 ('by', -0.30799901485443115),
 ('her', -0.30958837270736694)]

In [18]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 3.67569804e-01  1.45447865e-01 -7.67101124e-02 -2.51713127e-01
  1.62422106e-01  1.22999683e-01  3.16883624e-01 -7.20506757e-02
 -1.37708694e-01 -4.76133525e-01  2.02128574e-01 -9.18362383e-03
 -3.43216956e-02 -1.00194804e-01 -2.02343076e-01 -5.15149295e-01
  6.79034889e-02  3.50116134e-01  8.56724605e-02 -1.44431770e-01
  1.31192687e-03 -9.78999510e-02  3.70407701e-01 -7.84190223e-02
  1.03319041e-01  1.03175700e-01 -1.42763210e-02 -1.18491985e-01
 -1.24005981e-01 -3.48519742e-01  3.47845495e-01 -1.84158295e-01
  6.09976828e-01 -2.27926701e-01  1.24051876e-01  6.42717704e-02
 -1.44265875e-01 -9.54020470e-02 -3.52312744e-01 -5.64634055e-02
 -2.35963628e-01 -1.88571841e-01  1.49016291e-01 -2.58168608e-01
 -1.44456685e-01  2.92920232e-01  4.46063787e-01 -4.99566384e-02
 -1.44531220e-01  1.26240086e-02 -1.69196744e-02  3.29350948e-01
  2.33842768e-02  2.89818645e-01 -2.97025204e-01 -2.29905173e-01
 -8.86470526e-02  3.96068543e-02 -3.30910124e-02 -1.32381439e-01
 -2.89389133e-01  3.56112

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('ladies', 0.7097368240356445),
 ('showed', 0.6670942306518555),
 ('lie', 0.6648339629173279),
 ('em', 0.6454949378967285),
 ('because', 0.6437930464744568),
 ('hate', 0.6215324401855469),
 ('kicking', 0.6173309087753296),
 ('toe', 0.6169049739837646),
 ('dad', 0.6160704493522644)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('ladies', 0.7097368240356445),
 ('showed', 0.6670941710472107),
 ('lie', 0.6648339629173279),
 ('em', 0.6454949378967285),
 ('because', 0.6437930464744568),
 ('hate', 0.6215324401855469),
 ('kicking', 0.6173309087753296),
 ('toe', 0.6169049739837646),
 ('dad', 0.6160704493522644),
 ('crying', 0.6160150170326233)]

### 5 - Visualizar agrupación de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [20]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()

In [22]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show()

In [23]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.